In [ ]:
import mpmath
import numpy as np
from scipy.constants import h, k, e, pi, N_A
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy.integrate import quad
import pickle

# mpmathの精度を設定
mpmath.mp.dps = 200  # 小数点以下50桁の精度で計算

# 物理定数の定義
hbar = mpmath.mpf(h) / (2 * mpmath.pi)  # 換等プランク定数 (Js)
m_star_e = mpmath.mpf(1.4 * 9.11e-31)  # 電子の有効質量_e (kg)
m_star_h = mpmath.mpf(1.4 * 9.11e-31)  # 正孔の有効質量_h (kg)
kb = mpmath.mpf(k)  # ボルツマン定数 (J/K)
q = mpmath.mpf(e)  # 電子の電荷 (C)
T = mpmath.mpf(300)  # 温度 (K)
E_g = mpmath.mpf(0.1) * q  # バンドギャップ [eV]を[J]に変換
N_A = N_A  # アボガドロ数 (1/mol)
def xi_g(T): return E_g / (kb * T)

In [ ]:
# prompt: google drive に接続

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/MyDrive/T_range.pkl', 'rb') as f:
    T_range = pickle.load(f)
with open('/content/drive/MyDrive/N_D_values.pkl', 'rb') as f:
    N_D_values = pickle.load(f)
with open('/content/drive/MyDrive/xi_F_vals.pkl', 'rb') as f:
    xi_F_vals = pickle.load(f)

In [ ]:
xi_F_range = [mpmath.mpf(x) for x in np.linspace(-20, 20, 100)]

In [ ]:
print(T_range[22])
print([float(xi_F_vals[i][22]) for i in range(5)])

In [ ]:
print(len(N_D_values),len(T_range),len(xi_F_vals),len(xi_F_vals[0]))

In [ ]:
# N_C, N_Vの計算（探索コードに基づく追加部分）
N_C = 2 * ((m_star_e * kb * T) / (2 * mpmath.pi * hbar**2))**(3/2)
N_V = 2 * ((m_star_h * kb * T) / (2 * mpmath.pi * hbar**2))**(3/2)
E_D = mpmath.mpf(0.0261) * q  # ドナー準位 [eV]を[J]に変換
g_c = mpmath.mpf(2)  # 縮逆係数
g_v = mpmath.mpf(4)  # 縮逆係数

In [ ]:
def fermi_dirac(s, xi_F):
  def integrand(x): return x**s/(mpmath.exp(x-xi_F)+1)
  return mpmath.quad(integrand, [0, mpmath.inf])

In [ ]:
# 入力パラメータ y の更新
y=mpmath.mpf(0.8) # 入力パラメータ yの例

# parameters 定数
beta=mpmath.mpf(2.0)   # 正常過程とウムクラップ過程の緩和時間の比
gamma=mpmath.mpf(0.91) # グリュナイゼン定数
E=mpmath.mpf(2.94*e)   # 音響フォノン変形ポテンシャル定数

def a_cubed(y): return mpmath.power(2.7155e-10,3)*(1-y)+mpmath.power(2.8288e-10,3)*y # 平均原子体積
def a(y): return mpmath.power(a_cubed(y),1/3)  # 平均原子の３乗根
def M_g(y): return ((28.086)*(1-y)+(72.59)*y)  # 原子の平均質量 [g]
def M_kg(y): return M_g(y)*(1e-3)   # 原子の平均質量 [kg]
def G(y): return ((1.033)*(1-y)+(1.017)*y)*(1e-3) # 弾性定数
def Theta(y): return (1.48e-8)*a(y)**(-3/2)*(M_g(y)**(-1/2))*G(y) # デバイ温度 [K]
def v_s(y): return (kb/hbar)*(6*mpmath.pi**2)**(-1/3)*Theta(y)*a(y)  # 平均音速 [m/s]
def rho_d(y): return M_kg(y)/(a_cubed(y)*N_A) # 物質の密度 [kg/m^3]

print("y=", y)
print("a_cubed(y):", a_cubed(y))
print("a(y):", a(y))
print("M_g(y):", M_g(y))
print("M_kg(y):", M_kg(y))
print("G(y):", G(y))
print("Theta(y):", Theta(y))
print("v_s(y):",v_s(y))
print("rho_d(y):",rho_d(y))

# 1/τ_N の定義
def tau_N_inv(xi,T):
    # xi は積分中で変動する値
    term1 = ((20*mpmath.pi)/3)*hbar*N_A*((6*mpmath.pi**2)/4)**(1/3)*(beta*(1+(5/9)*beta)/(1+beta))*(gamma**2)/(M_kg(y)*a(y)**2)*(T/Theta(y))**3*(xi**2)
    return term1
def tau_N(xi,T):
    return 1/tau_N_inv(xi,T)

# 1/τ_U の定義
def tau_U_inv(y, xi, T):
    tau_N_inv_val = tau_N_inv(xi, T)
    return (1 / beta) * tau_N_inv_val

# 1/τ_C の計算
def tau_C_inv(xi, xi_F, T):
    tau_N_inv_val = tau_N_inv(xi, T)
    tau_U_inv_val = tau_U_inv(y,xi,T)
    return tau_N_inv_val + tau_U_inv_val

# τ_C を積分中で動的に計算
def tau_C(xi, xi_F, T):
    tau_C_val = 1 / tau_C_inv(xi, xi_F, T)
    return tau_C_val

def tau_integrand(x): return mpmath.power(x,4)*mpmath.exp(x)/mpmath.power(mpmath.expm1(x),2)

# I1の積分
def I1(xi_F, T):
    def I1_integrand(xi):
      return tau_C(xi,xi_F,T)*tau_integrand(xi)
    return mpmath.quad(I1_integrand, [0, Theta(y) / T])

# I2の積分
def I2(xi_F, T):
    def I2_integrand(xi):
      return tau_C(xi,xi_F,T)*(tau_N_inv(xi,T))*tau_integrand(xi)
    return mpmath.quad(I2_integrand, [0, Theta(y) / T])

# I3の積分
def I3(xi_F, T):
    def I3_integrand(xi):
      return tau_N_inv(xi,T) * (1-(tau_C(xi,xi_F,T)/tau_N(xi,T))) * tau_integrand(xi)
    return mpmath.quad(I3_integrand, [0, Theta(y) / T])


In [ ]:
# 1/τ_dp の定義(σで利用)
def tau_dp_e(y,T):
    term1 = mpmath.sqrt(2)*E**2*(m_star_e*kb*T)**(3/2)
    term2 = mpmath.pi*rho_d(y)*hbar**4*v_s(y)**2
    return (term2/term1)

def tau_dp_h(y,T):
    term1 = mpmath.sqrt(2)*E**2*(m_star_h*kb*T)**(3/2)
    term2 = mpmath.pi*rho_d(y)*hbar**4*v_s(y)**2
    return (term2/term1)

print("tau_dp_inv_e:",tau_dp_e(y,T))
print("tau_dp_inv_h:",tau_dp_h(y,T))

In [ ]:
def delta(s, xi_F): return (s+5/2) * fermi_dirac(s+3/2, xi_F)/((s+3/2) * fermi_dirac(s+1/2, xi_F))
def Delta(s, xi_F): return (s+7/2) * fermi_dirac(s+5/2, xi_F)/((s+3/2) * fermi_dirac(s+1/2, xi_F))-delta(s, xi_F)**2

def sigma1(s, xi_F, N_B, m_star, tau_val): return ((4 * q**2 * N_B * tau_val)/(3 * mpmath.sqrt(pi) * m_star)) * (s+3/2) * fermi_dirac(s+1/2, xi_F) # 電気伝導率
def alpha1(s, xi_F): return (kb/q)*(delta(s,xi_F)-xi_F) # ゼーベック係数
def L1(s,xi_F): return (kb/q)**2*Delta(s,xi_F) #ローレンツ数

In [ ]:
def N_B(m_star,T): return 2 * (m_star * kb * T /(2*pi*hbar**2))**(3/2)
def N_C(T): return N_B(m_star_e,T)
def N_V(T): return N_B(m_star_h,T)

def alpha2e(s,xi_F): return -alpha1(s,xi_F)
def alpha2h(s,xi_F,T): return alpha1(s,-xi_F-xi_g(T))
def sigma2e(s,xi_F,T): return sigma1(s,xi_F,N_C(T), m_star_e,tau_dp_e(y,T))
def sigma2h(s,xi_F,T): return sigma1(s,-xi_F-xi_g(T),N_V(T), m_star_h,tau_dp_h(y,T))
def L2e(s,xi_F): return L1(s,xi_F)
def L2h(s,xi_F,T): return L1(s,-xi_F-xi_g(T))
def kappa2e(s,xi_F,T,y=0.8): return L2(s,xi_F,T)*T*sigma2(s,xi_F,T)
def kappa2L(s,xi_F,T,y=0.8): return (kb / (2 * mpmath.pi ** 2 * v_s(y))) * ((kb * T) / hbar) ** 3 * (I1(xi_F, T) + (I2(xi_F, T) ** 2) / I3(xi_F, T))

def sigma2(s,xi_F,T): return sigma2e(s,xi_F,T)+sigma2h(s,xi_F,T)
def alpha2(s,xi_F,T): return (alpha2e(s,xi_F)*sigma2e(s,xi_F,T)+alpha2h(s,xi_F,T)*sigma2h(s,xi_F,T))/sigma2(s,xi_F,T)
def L2(s,xi_F,T): return ((L2e(s,xi_F)*sigma2e(s,xi_F,T)+L2h(s,xi_F,T)*sigma2h(s,xi_F,T))/sigma2(s,xi_F,T))+((sigma2e(s,xi_F,T)*sigma2h(s,xi_F,T)*(alpha2e(s,xi_F)-alpha2h(s,xi_F,T))**2)/sigma2(s,xi_F,T)**2)
def kappa2(s,xi_F, T, y=0.8): return kappa2e(s,xi_F,T,y)+kappa2L(s,xi_F,T,y)
def ZT2(alpha2_val,sigma2_val,kappa2_val,T): return alpha2_val**2*sigma2_val*T/kappa2_val

In [ ]:
alpha2_val = [[alpha2(-1/2, xi_F_range[j], T) for j in range(len(xi_F_range))]
               for i in range(len(N_D_values))]

In [ ]:
sigma2_val = [[sigma2(-1/2, xi_F_range[j], T) for j in range(len(xi_F_range))]
               for i in range(len(N_D_values))]

In [ ]:
L2_val     = [[L2(-1/2, xi_F_range[j], T) for j in range(len(xi_F_range))]
               for i in range(len(N_D_values))]

In [ ]:
kappa2_val = [[kappa2(-1/2, xi_F_range[j], T) for j in range(len(xi_F_range))]
               for i in range(len(N_D_values))]

In [ ]:
ZT2_val    = [[ZT2(alpha2_val[i][j], sigma2_val[i][j], kappa2_val[i][j], T)
               for j in range(len(xi_F_range))]
               for i in range(len(N_D_values))]

In [ ]:
# σのグラフ
# モノクロで判別可能な線種リスト
line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]

plt.figure(figsize=(10, 6))
for i, N_D in enumerate(N_D_values):
    ls = line_styles[i % len(line_styles)]
    # 横軸は xi_F の値（各 N_D に対する xi_F_vals[i]）
    plt.plot(xi_F_vals[i], sigma2_val[i],
             label=f"N_D = {N_D:1.2e}",
             linestyle=ls,
             linewidth=2)
plt.axhline(0, color="gray", linestyle=":")
plt.xlabel("ξ_F", fontsize=18)
plt.ylabel("Electrical conductivity σ [S/m]", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale("log")
plt.xlim(-20, 20)
plt.legend(fontsize=12)
plt.grid(True, which="both", linestyle=":")
plt.show()

In [ ]:
# グラフ: alpha



line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]

plt.figure(figsize=(10, 6))
for i, N_D in enumerate(N_D_values):
    ls = line_styles[i % len(line_styles)]
    plt.plot(xi_F_vals[i], np.array(alpha2_val[i]) * 1000,  # 単位変換 (V/K → mV/K)
             label=f"N_D = {N_D:1.2e}",
             linestyle=ls,
             linewidth=2)
plt.axhline(0, color="gray", linestyle=":")
plt.xlabel("ξ_F", fontsize=18)
plt.ylabel("Seebeck coefficient α [mV/K]", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=12)
plt.xlim(-20, 20)
plt.grid(True, which="both", linestyle=":")
plt.show()


In [ ]:
# グラフ: L

line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]

plt.figure(figsize=(10, 6))
for i, N_D in enumerate(N_D_values):
    ls = line_styles[i % len(line_styles)]
    plt.plot(xi_F_vals[i], L2_val[i],
             label=f"N_D = {N_D:1.2e}",
             linestyle=ls,
             linewidth=2)
plt.axhline(0, color="gray", linestyle=":")
plt.xlabel("ξ_F", fontsize=18)
plt.ylabel("Lorenz number L [WΩ/K²]", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=12)
plt.xlim(-20, 20)
plt.grid(True)
plt.show()

In [ ]:
# グラフ: kappa

# T_range を 100 個の等間隔の値にする
#T_range = np.linspace(300, 1300, 100) #これでやるとκ_Lになる

plt.figure(figsize=(10, 6))
for i, N_D in enumerate(N_D_values):
    ls = line_styles[i % len(line_styles)]
    plt.plot(xi_F_vals[i], kappa2_val[i],
             label=f"N_D = {N_D:1.2e}",
             linestyle=ls,
             linewidth=2)
plt.axhline(0, color="gray", linestyle=":")
plt.xlabel("ξ_F", fontsize=18)
plt.ylabel("Thermal conductivity κ [W/mK]", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=12)
plt.xlim(-20, 20)
plt.grid(True, which="both", linestyle=":")
plt.show()

In [ ]:
# グラフ: ZT



line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]

plt.figure(figsize=(10, 6))
for i, N_D in enumerate(N_D_values):
    ls = line_styles[i % len(line_styles)]
    plt.plot(xi_F_vals[i], ZT2_val[i],
             label=f"N_D = {N_D:1.2e}",
             linestyle=ls,
             linewidth=2)
plt.xlabel("ξ_F", fontsize=18)
plt.ylabel("ZT", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=12)
plt.xlim(-20, 20)
plt.grid(True, which="both", linestyle=":")
plt.show()
